In [10]:
#importing libararies
import pandas as pd
import numpy as np
import os
from __future__ import print_function
from mailmerge import MailMerge
from datetime import date
import docx
import import_ipynb
import glob

import warnings
warnings.filterwarnings("ignore")


def format_sheet(file_path:str,template_path:str,sheet_num_template=0,run=0):
    """
    file_path: the file you want to design, must bx xlsx
    template_path: the teplate of the design, must bx xlsx
        """
    # format sheet
    xlsx=pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    for i in range(len(xlsx.book.worksheets)):
        new_sheet=xlsx.book.worksheets[run]
        template=pd.ExcelWriter(template_path,engine='openpyxl', mode='a')
        default_sheet=template.book.worksheets[sheet_num_template]
        from copy import copy
        new_sheet.sheet_view.rightToLeft=True
        for row in default_sheet.rows:
            for cell in row:
                new_cell=new_sheet.cell(row=cell.row, column=cell.col_idx)
                if cell.has_style and new_cell.value != None:
                    new_cell.font=copy(cell.font)
                    new_cell.border=copy(cell.border)
                    new_cell.fill=copy(cell.fill)
                    new_cell.number_format=copy(cell.number_format)
                    new_cell.protection=copy(cell.protection)
                    new_cell.alignment=copy(cell.alignment)
        from openpyxl.utils import get_column_letter
        for i in range(default_sheet.max_column):
            new_sheet.column_dimensions[get_column_letter(i + 1)].width=default_sheet.column_dimensions[
                get_column_letter(i + 1)].width
    
    xlsx.save()
def change_names_and_order(file_path:str,df:pd.DataFrame):
    """
    name of columns in names must be "old_name", "new_name"
        """
    match_names=pd.read_excel(file_path)
    match_names.index=match_names.old_name
    dict_names=match_names.to_dict()['new_name']
    ls_names=match_names['new_name'].tolist()
    df.rename(columns=dict_names, inplace=True)
    df=df[ls_names]
    return df


In [21]:
#setting the location as project folder
file_directory = open('directory.txt','r')
location = file_directory.read()

perm_file_directory = open('perma directory.txt','r')
prem_location = perm_file_directory.read()

template_directory = open('template directory.txt','r')
template_location = template_directory.read()

#reading the check file
filepath_check = location+"\*דיווחים*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    check = pd.read_excel(textfile)



#reading the risk country file
filepath_check = prem_location+"\*מדינות*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rc = pd.read_excel(textfile)

#reading the rates file
filepath_check = prem_location+"\*שערים*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rates = pd.read_excel(textfile ,sheet_name="CUR CONV",usecols="F:G",skiprows=2)
    

#reading the report number generator file
filepath_check = template_location+"\*מחולל*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    report_num = pd.read_excel(textfile)
    
#reading the report content file
filepath_check = template_location+"\*תוכן*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    content = pd.read_excel(textfile)    
    
#a guide to report types: p-pitzul , d - dirdos , m - mosheh , r - risk

In [22]:
#getting the data ready
#naming rates columns
rates.columns=["curr","rate"]

#adding the rates value
check= check.merge(rates, how = 'left', left_on = 'Send currency', right_on = 'curr')
check['nis sum'] = check['rate'] * check['Remittance amount']

#lower the country cases:
check['Issuance country'] = check['Issuance country'].str.lower()
check['Destination country'] = check['Destination country'].str.lower()
check['Source country'] = check['Source country'].str.lower()
rc['English short name (using title case)'] = rc['English short name (using title case)'].str.lower()
check = check.fillna('')

#risk check
check = check.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']] , left_on = 'Source country', right_on = 'English short name (using title case)',how = 'left',suffixes = ('','_source'))
check = check.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']] , left_on = 'Destination country', right_on = 'English short name (using title case)',how = 'left',suffixes = ('','_dest'))
check = check.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']] , left_on = 'Issuance country', right_on = 'English short name (using title case)',how = 'left',suffixes = ('','_issuance'))

#outbound or inbound
outbound = check[check['Type'] == 'SEND']
inbound = check[check['Type'] == 'PAYOUT']

#check risk 
check_risk = check[(check['סיכון/ללא דיווח רגיל'].notna()) | (check['סיכון/ללא דיווח רגיל_dest'].notna()) | (check['סיכון/ללא דיווח רגיל_issuance'].notna())]

In [23]:
#building the monitoring df with sum per client, deal distinct count and under 50k deal count
outbound_monitoring = outbound.groupby('Sender').agg({'nis sum':'sum','Reservation code':'nunique'})
outbound_monitoring = outbound_monitoring.reset_index()

#sorting the values by sum
outbound_monitoring = outbound_monitoring.sort_values('nis sum',ascending = False)

#filter only client over 47K
outbound_monitoring = outbound_monitoring[outbound_monitoring['nis sum'] >= 47000]

#adding a column named status based on unique count of deal number and at least on tr under 50k and not reported in the past 2m
outbound_monitoring = outbound_monitoring[((outbound_monitoring['Reservation code'] > 1) & (outbound_monitoring['nis sum'] >= 50000))|(outbound_monitoring['nis sum'] < 50000)]

#marking dirdos transction as d in a new column named dirdos
outbound_monitoring['type'] = np.where((outbound_monitoring['nis sum']>= 47000) & (outbound_monitoring['nis sum']<50000) , "d",'p')
outbound_monitoring=outbound_monitoring.reset_index()

In [24]:
#out bound risk
outbound_risk = outbound[(outbound['סיכון/ללא דיווח רגיל'].notna()) | (outbound['סיכון/ללא דיווח רגיל_dest'].notna()) | (outbound['סיכון/ללא דיווח רגיל_issuance'].notna())]

#outbound tisk monitorng df
outbound_risk_monitoring = outbound_risk.groupby('Sender').agg({'nis sum':'sum','Reservation code':'nunique'})

#sorting the values by sum
outbound_risk_monitoring = outbound_risk_monitoring.sort_values('nis sum',ascending = False)

#filtering to report tr
outbound_risk_monitoring = outbound_risk_monitoring[outbound_risk_monitoring['nis sum'] >= 4700]
outbound_risk_monitoring = outbound_risk_monitoring[((outbound_risk_monitoring['Reservation code'] > 1) & (outbound_risk_monitoring['nis sum'] >= 5000)) | (outbound_risk_monitoring['nis sum'] < 5000)]


#adding the type of the report pr-pitzul dr-dirdos (risk)
outbound_risk_monitoring['type'] =  np.where((outbound_risk_monitoring['nis sum'] >= 5000), "pr" , "dr")
outbound_risk_monitoring = outbound_risk_monitoring.reset_index()

In [25]:
try:
    report = outbound_risk_monitoring[['type','Sender']].merge(outbound_monitoring[['type','Sender']], on = 'Sender',how = 'outer',suffixes = ('_risk',''))
except KeyError:
    report = outbound_risk_monitoring[['type','Sender']]


In [26]:
#building the monitoring df with sum per client and deal distinct count
outbound_ow = outbound.groupby(['Receiver']).agg({'nis sum':'sum','Reservation code':'nunique','Sender':'nunique'})

#sorting the values by sum
outbound_ow = outbound_ow.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
outbound_ow = outbound_ow[(outbound_ow['Reservation code'] > 1) & (outbound_ow['Sender'] > 1) & (outbound_ow['nis sum'] >= 50000)]
outbound_ow = outbound_ow.reset_index()

#creating a list of the shared owner report clients name
outbound_ow = outbound_ow.merge(outbound[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
outbound_ow = outbound_ow.merge(report, on ='Sender' , how = 'left', suffixes = ('','_report'))


try:
    outbound_ow = outbound_ow[ (outbound_ow['type'].isna()) & (outbound_ow['type_risk'].isna())]
except KeyError:
    outbound_ow =  outbound_ow
        
outbound_ow['type_m'] = 'm'
outbound_ow = outbound_ow.drop_duplicates('Receiver')
report = outbound_ow[['Sender','type_m']].merge(report, on = 'Sender', how = 'outer')

In [27]:
#building the monitoring df with sum per client and deal distinct count risk
outbound_ow_risk = outbound_risk.groupby(['Receiver']).agg({'nis sum':'sum','Reservation code':'nunique','Sender':'nunique'})

#sorting the values by sum
outbound_ow_risk = outbound_ow_risk.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
outbound_ow_risk = outbound_ow_risk[(outbound_ow_risk['Reservation code'] > 1) & (outbound_ow_risk['Sender'] > 1) & (outbound_ow_risk['nis sum'] >= 5000)]
outbound_ow_risk = outbound_ow_risk.reset_index()

#creating a list of the shared owner report clients name____not ready yet!!!!
outbound_ow_risk = outbound_ow_risk.merge(outbound_risk[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
outbound_ow_risk = outbound_ow_risk.merge(report, on ='Sender' , how = 'left')

try:
    outbound_ow_risk = outbound_ow_risk[(outbound_ow_risk['type'].isna()) & (outbound_ow_risk['type_risk'].isna()) & (outbound_ow_risk['type_m'].isna())]
except KeyError:
    outbound_ow_risk =  outbound_ow_risk

outbound_ow_risk['type_m_risk'] = 'mr'
outbound_ow_risk = outbound_ow_risk.drop_duplicates('Receiver')

#merging the risk ow with regular report
try:
    report = outbound_ow_risk[['Sender','type_m_risk']].merge(report, on = 'Sender', how = 'outer')
except KeyError:
    report = report

In [28]:
#building the monitoring df with sum per client, deal distinct count and under 50k deal count
inbound_monitoring = inbound.groupby('Receiver').agg({'nis sum':'sum','Reservation code':'nunique'})
inbound_monitoring = inbound_monitoring.reset_index()

#sorting the values by sum
inbound_monitoring = inbound_monitoring.sort_values('nis sum',ascending = False)

#filter only client over 47K
inbound_monitoring = inbound_monitoring[inbound_monitoring['nis sum'] >= 47000]

#adding a column named status based on unique count of deal number and at least on tr under 50k and not reported in the past 2m
inbound_monitoring = inbound_monitoring[((inbound_monitoring['Reservation code'] > 1) & (inbound_monitoring['nis sum'] >= 50000)) | (inbound_monitoring['nis sum'] < 50000)]

#marking dirdos transction as d in a new column named dirdos
inbound_monitoring['type'] = np.where((inbound_monitoring['nis sum']>= 47000) & (inbound_monitoring['nis sum']<50000) , "kd",'kp')
inbound_monitoring=inbound_monitoring.reset_index()

In [29]:
#out bound risk
inbound_risk = inbound[(inbound['סיכון/ללא דיווח רגיל'].notna()) | (inbound['סיכון/ללא דיווח רגיל_dest'].notna()) | (inbound['סיכון/ללא דיווח רגיל_issuance'].notna())]

         
#outbound tisk monitorng df
inbound_risk_monitoring = inbound_risk.groupby('Receiver').agg({'nis sum':'sum','Reservation code':'nunique'})

#filtering to report tr
inbound_risk_monitoring = inbound_risk_monitoring[(inbound_risk_monitoring['nis sum'] >= 4700)]
inbound_risk_monitoring = inbound_risk_monitoring[((inbound_risk_monitoring['Reservation code'] > 1) & (inbound_risk_monitoring['nis sum'] >= 5000)) | (inbound_risk_monitoring['nis sum'] < 5000)]


#adding the type of the report pr-pitzul dr-dirdos (risk)
inbound_risk_monitoring['type_risk'] =  np.where((inbound_risk_monitoring['nis sum'] >= 5000), "kpr" , "kdr")
inbound_risk_monitoring = inbound_risk_monitoring.reset_index()

#rename the columns
#inbound_risk_monitoring = inbound_risk_monitoring.rename(columns = {'owner name' = 'clients name'})

In [30]:
try:
    inbound_report = inbound_risk_monitoring[['type_risk','Receiver']].merge(inbound_monitoring[['type','Receiver']], on = 'Receiver',how = 'outer')
except KeyError:
    inbound_report = inbound_monitoring[['Receiver','type']]


In [31]:
#building the monitoring df with sum per client and deal distinct count
inbound_ow = inbound.groupby(['Sender']).agg({'nis sum':'sum','Reservation code':'nunique','Receiver':'nunique'})

#sorting the values by sum
inbound_ow = inbound_ow.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
inbound_ow = inbound_ow[(inbound_ow['Reservation code'] > 1) & (inbound_ow['Receiver'] > 1) & (inbound_ow['nis sum'] >= 50000)]
inbound_ow = inbound_ow.reset_index()

#creating a list of the shared owner report clients name____not ready yet!!!!
inbound_ow = inbound_ow.merge(inbound[['Sender','Receiver']], on = 'Sender', how = 'left',suffixes = ('_count','') )


inbound_ow = inbound_ow.merge(inbound_report, on ='Receiver' , how = 'left')

try:
    inbound_ow = inbound_ow[inbound_ow['type'].isna()]
except KeyError:
    inbound_ow =  inbound_ow
        
inbound_ow['type_mk'] = 'mk'
inbound_ow = inbound_ow.drop_duplicates('Sender')

try:   
    inbound_report = inbound_ow[['Receiver','type_mk']].merge(inbound_report, on = 'Receiver', how = 'outer')
except KeyError :
    inbound_report = inbound_report

In [32]:
#building the monitoring df with sum per client and deal distinct count risk
inbound_ow_risk = inbound_risk.groupby(['Sender']).agg({'nis sum':'sum','Reservation code':'nunique','Receiver':'nunique'})

#sorting the values by sum
inbound_ow_risk = inbound_ow_risk.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
inbound_ow_risk = inbound_ow_risk[(inbound_ow_risk['Reservation code'] > 1) & (inbound_ow_risk['Receiver'] > 1) & (inbound_ow_risk['nis sum'] >= 5000)]
inbound_ow_risk = inbound_ow_risk.reset_index()

#creating a list of the shared owner report clients name____not ready yet!!!!
inbound_ow_risk = inbound_ow_risk.merge(inbound_risk[['Sender','Receiver']], on = 'Sender', how = 'left',suffixes = ('_count','') )
inbound_ow_risk = inbound_ow_risk.merge(inbound_report, on ='Receiver' , how = 'left')


try:
    inbound_ow_risk = inbound_ow_risk[(inbound_ow_risk['type'].isna()) & (inbound_ow_risk['type_mk'].isna())]
except KeyError:
    inbound_ow_risk =  inbound_ow_risk
        
inbound_ow_risk['type_mkr'] = 'mkr'
inbound_ow_risk = inbound_ow_risk.drop_duplicates('Sender')

#merging the risk ow with regular report
try:
    inbound_report = inbound_ow_risk[['Receiver','type_mkr']].merge(inbound_report , on = 'Receiver', how = 'outer')
except KeyError:
    inbound_report = inbound_report


In [33]:
#creating the details to report inbound and outbound
#outbound
report = report.merge(outbound[['Sender','Destination country','Receiver','Sender date of birth','Sender city','Sender street address','User id number','Issuance country','Sender gender','User id type']],on = 'Sender',how = 'left')
report = report.drop_duplicates('Sender')
report = report.fillna('')
report['final type'] = report['type']+report['type_risk']+report['type_m'] + report['type_m_risk']

#inbound
inbound_report = inbound_report.merge(inbound[['Receiver','Source country','Sender date of birth','Sender city','Sender street address','User id number','Issuance country','Sender gender','User id type']],on = 'Receiver',how = 'left')
inbound_report = inbound_report.drop_duplicates('Receiver')
inbound_report = inbound_report.fillna('')
inbound_report['final type'] = inbound_report['type']+inbound_report['type_risk'] + inbound_report['type_mk'] + inbound_report['type_mkr']

In [34]:
inbound_report = inbound_report.rename(columns = {'Receiver':'Sender','Source country':'Destination country'})
inbound_report= inbound_report.reset_index(drop=True)
report = report.reset_index(drop=True)

full_report = report.append(inbound_report)

In [35]:
full_report = full_report.fillna('')
full_report

,Sender,type_m_risk,type_m,type_risk,type,Destination country,Receiver,Sender date of birth,Sender city,Sender street address,User id number,Issuance country,Sender gender,User id type,final type,type_mkr,type_mk
0,Kumar Tamang,mr,,,,cambodia,SkuN DarA,1980-10-08,Kadima Zoran,Mam Gimal St 23,m8841960,india,M,PASSPORT,mr,,
1,Samah Elkarnawi,mr,,,,turkey,DHEKRA OUNISSA,1991-02-11,RAHAT,ELNAJAH18,203512470,israel,M,ID,mr,,
2,Dorisa Rivkin,mr,,,,turkey,Sevcan Demircan,1939-07-24,beer shva,asaf 18,312648850,israel,M,ID,mr,,
3,Tufan Domurcuk,mr,,,,turkey,Senem Domurcuk,1984-08-02,telaviv,levinsky 18,u23396510,turkey,M,PASSPORT,mr,,
4,Mariana Nikolaevski,mr,,,,turkey,AllA MelniC,1971-12-27,ASHKELON,SD BEN GURION 16,306576497,israel,F,ID,mr,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Natalya Vengrover,,,,kd,russian federation,,,,,346563661,,,ID,kd,,
42,Mark Iznyuk,,,,kd,russian federation,,,,,346866361,,,ID,kd,,
43,Sergey Blinov,,,,kd,russian federation,,,,,303863401,,,ID,kd,,
44,Marina Sivkova,,,,kd,russian federation,,,,,9562662,,,PASSPORT,kd,,


In [36]:
#importing the report number needed
#report_num = pd.read_excel(location+r'\מחולל שם דיווח- UNISTREAM.xlsx')
report_id = (report_num.iloc[1,5] + 1).astype(int)
full_report['report_id'] = report_id + range(len(full_report.index))
full_report['report_name'] = report_num.iloc[3,1].rsplit('-',1)[0]

In [37]:
#checking the risk type
risk_country4report = full_report.merge(rc[['English short name (using title case)','עברית','סיכון/ללא דיווח רגיל']], left_on = 'Issuance country', right_on ='English short name (using title case)', how = 'left')
risk_country4report = risk_country4report.merge(rc[['English short name (using title case)','עברית','סיכון/ללא דיווח רגיל']], left_on = 'Destination country', right_on ='English short name (using title case)', how = 'left',suffixes = ('','_dest'))



In [38]:
#creating a column with the hebrew name of the country for risk report
risk_country4report['risk_content'] = np.where((risk_country4report['סיכון/ללא דיווח רגיל'].notna()) & ((risk_country4report['final type'] != 'm') | (risk_country4report['final type'] != 'p') | (risk_country4report['final type'] != 'd')) ,risk_country4report['עברית'],risk_country4report['עברית_dest'])


In [39]:
#merging the report content
risk_country4report = risk_country4report.merge(content, left_on = 'final type' , right_on = 'type', how = 'left')
risk_country4report['risk_full_content'] ='הכספים נשלחו ל'+risk_country4report['risk_content'] + ' ,מדינה המוגדרת בסיכון גבוה.'


In [40]:
risk_country4report['risk_full_content'] =np.where((risk_country4report['final type'] == 'mr')|(risk_country4report['final type'] == 'pr')|(risk_country4report['final type'] == 'dr')|(risk_country4report['final type'] == 'mkr')|(risk_country4report['final type'] == 'kpr')|(risk_country4report['final type'] == 'kdr') ,'הכספים נשלחו ל'+risk_country4report['risk_content'] + ' ,מדינה המוגדרת בסיכון גבוה.','')


In [41]:
final_client2report = risk_country4report[['risk_full_content','Content','Title','עברית_dest','עברית','report_name','report_id','final type','User id number','Sender street address','Sender city','Sender date of birth','Sender','Sender gender','User id type' ]]
final_client2report['Sender gender'] = final_client2report['Sender gender'].map({'F':'נקבה','M':'זכר'})
final_client2report = final_client2report.fillna('')
final_client2report.to_excel("monitoring.xlsx")

In [44]:
template

'G:/.shortcut-targets-by-id/0ByI8FTktHVHuRmhTcUhDUVNiUkk/ריטיינר דיווחים/ש.ט.ב. יוניון בעמ\\~$ווח טמפלייט חדש.docx'

In [45]:
#reading the template file
#filepath_check = template_location+"\*טמפלייט*"
#txt_check = glob.glob(filepath_check)
#for textfile in txt_check:
#    before_template = textfile
#template = before_template
template = template_location+r'\דיווח טמפלייט חדש.docx'
document = MailMerge(template)
print(document.get_merge_fields())

{'כותרת', 'עיר', 'תוכן', 'מין', 'סיכון', 'כתובת', 'אזרחות', 'תאריך_לידה', 'שם_הלקוח', 'מס_דיווח', 'סוג_תעודה_מזהה', 'תז'}


In [46]:

for i in final_client2report.index:
     #template1 = 'test1.docx'
    document = MailMerge(template)
    document.merge(
        שם_הלקוח = str(final_client2report['Sender'][i]),
        מין = str(final_client2report['Sender gender'][i]),
        תאריך_לידה = str(final_client2report['Sender date of birth'][i]),
        כותרת = str(final_client2report['Title'][i]),
        מס_דיווח = str(final_client2report['report_id'][i]),
        תוכן = str(final_client2report['Content'][i]),
        סיכון = str(final_client2report['risk_full_content'][i]),
        אזרחות = str(final_client2report['עברית'][i]),
        עיר = str(final_client2report['Sender city'][i]),
        תז = str(final_client2report['User id number'][i]),
        כתובת = str(final_client2report['Sender street address'][i]),
        סוג_תעודה_מזהה = str(final_client2report['User id type'][i] ))
    

    output =  str(final_client2report['report_name'][i]) + "-" + str(final_client2report['report_id'][i]) + '.docx'
    document.write(output)

In [47]:
#filtering outbound reports
outbound_report = final_client2report[(final_client2report['final type'] == 'p') |(final_client2report['final type'] == 'd') |(final_client2report['final type'] == 'dr') |(final_client2report['final type'] == 'pr') | (final_client2report['final type'] == 'ppr')]

#creating a data frames with only need to report transctions
outbound_report_tr = check[check['Sender'].isin(outbound_report['Sender'])]

#leaving only the imporatant columns
outbound_report_tr = outbound_report_tr[['Reservation code','Date','Type','Destination country','Sender','Receiver','Remittance amount','Send currency','User id number','nis sum']]

#creating an excel shhet with the ountbound reports
i=0
for_word_table=[]
xlsx=pd.ExcelWriter(location+r"/report.xlsx", engine='openpyxl', mode='w')
#round the sum in ils
outbound_report_tr["nis sum"]=outbound_report_tr["nis sum"].round()

#creating a unique list of the sender names
ls_customer=outbound_report_tr["Sender"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer:
    df=outbound_report_tr[outbound_report_tr["Sender"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()

In [50]:
#indexing the outbound_report data frame
info2r = outbound_report.set_index('Sender')
info2r_nona = info2r[info2r.index.notnull()]

for title in ls_customer:
    
    data = outbound_report_tr[outbound_report_tr['Sender'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(),data['Remittance amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Sender", "nis sum",'Remittance amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(' ')
    
    orderd=change_names_and_order(location+r"/change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    
    doc = docx.Document(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")


In [51]:
#filtering only shared owner reports
outbound_report_ow = final_client2report[(final_client2report['final type'] == 'm') |(final_client2report['final type'] == 'mr')]

#concatacating the two kinds of outbound shared owner transction dataframes
outbound_tr_ow = pd.concat([outbound_ow_risk,outbound_ow])

#filtering only need to report transctions
outbound_report_tr_ow = check[check['Receiver'].isin(outbound_tr_ow['Receiver'])]

#leaving only the imporatant columns
outbound_report_tr_ow = outbound_report_tr_ow[['Reservation code','Date','Type','Destination country','Sender','Receiver','Remittance amount','Send currency','User id number','nis sum']]

#creating a new list
j=0
for_word_table_ow=[]

#wrting an excel file containing the report transctions for each customer
xlsx=pd.ExcelWriter(location+r"/report_ow.xlsx", engine='openpyxl', mode='w')

#round the sum in ils
outbound_report_tr_ow["nis sum"]=outbound_report_tr_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer_ow=outbound_report_tr_ow["Receiver"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer_ow:
    df=outbound_report_tr_ow[outbound_report_tr_ow["Receiver"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()

In [52]:
#creating a list containing the client name and owner name
info2r_ow = outbound_report_ow[['report_name','report_id','Sender']].merge(outbound_report_tr_ow[['Sender','Receiver']], on = 'Sender', how = 'left')
info2r_ow = info2r_ow.drop_duplicates('Receiver')
info2r_ow = info2r_ow.set_index('Receiver')


for title in ls_customer_ow:    
    data = outbound_report_tr_ow[outbound_report_tr_ow['Receiver'] == title]
    table = [["סהכ",data["nis sum"].sum().round(),data['Remittance amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Receiver", "nis sum",'Remittance amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(" ")
    orderd=change_names_and_order(location+r"/change_names.xlsx",orderd)
    
    doc = docx.Document(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")
    
       

In [53]:
#filtering outbound reports
inbound_report = final_client2report[(final_client2report['final type'] == 'kp') |(final_client2report['final type'] == 'kd') |(final_client2report['final type'] == 'kdr') |(final_client2report['final type'] == 'kpr')]

#creating a data frames with only need to report transctions
inbound_report_tr = check[check['Receiver'].isin(inbound_report['Sender'])]

#leaving only the imporatant columns
inbound_report_tr = inbound_report_tr[['Reservation code','Date','Type','Source country','Sender','Receiver','Remittance amount','Send currency','User id number','nis sum']]

#creating an excel shhet with the ountbound reports
i=0
for_word_table=[]
xlsx=pd.ExcelWriter(location+r"/report inbound.xlsx", engine='openpyxl', mode='w')
#round the sum in ils
inbound_report_tr["nis sum"]=inbound_report_tr["nis sum"].round()

#creating a unique list of the sender names
ls_customer=inbound_report_tr["Receiver"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer:
    df=inbound_report_tr[inbound_report_tr["Receiver"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Receiver", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()

In [54]:
#indexing the outbound_report data frame
info2r_inbound = inbound_report.set_index('Sender')
info2r_nona_inbound = info2r_inbound[info2r_inbound.index.notnull()]

for title in ls_customer:
    
    data = inbound_report_tr[inbound_report_tr['Receiver'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(),data['Remittance amount'].sum().round()]]
    last_row = pd.DataFrame(table, columns = ["Sender", "nis sum",'Remittance amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(' ')
    orderd = orderd.rename(columns = {'Sender':'Receiver1','Receiver':'Sender'})
    orderd = orderd.rename(columns = {'Receiver1':'Receiver','Source country':'Destination country'})
    orderd=change_names_and_order(location+r"/change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    
    doc = docx.Document(str(info2r_nona_inbound['report_name'].loc[title]) + "-" + str(info2r_nona_inbound['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona_inbound['report_name'].loc[title]) + "-" + str(info2r_nona_inbound['report_id'].loc[title]) + ".docx")


In [55]:
#filtering only shared owner reports
inbound_report_ow = final_client2report[(final_client2report['final type'] == 'mk') |(final_client2report['final type'] == 'mkr')]

#concatacating the two kinds of outbound shared owner transction dataframes
inbound_tr_ow = pd.concat([inbound_ow_risk,inbound_ow])

#filtering only need to report transctions
inbound_report_tr_ow = check[check['Sender'].isin(inbound_tr_ow['Sender'])]

#leaving only the imporatant columns
inbound_report_tr_ow = inbound_report_tr_ow[['Reservation code','Date','Type','Source country','Sender','Receiver','Remittance amount','Send currency','User id number','nis sum']]

#creating a new list
j=0
for_word_table_ow=[]

#wrting an excel file containing the report transctions for each customer
xlsx=pd.ExcelWriter(location+r"/report_ow_inbound.xlsx", engine='openpyxl', mode='w')

#round the sum in ils
inbound_report_tr_ow["nis sum"]=inbound_report_tr_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer_ow=inbound_report_tr_ow["Sender"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer_ow:
    df=inbound_report_tr_ow[inbound_report_tr_ow["Sender"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()

In [56]:
#creating a list containing the client name and owner name
inbound_report_ow = inbound_report_ow.rename(columns = {'Sender':'Receiver'})
info2r_ow_inbound = inbound_report_ow[['report_name','report_id','Receiver']].merge(inbound_report_tr_ow[['Sender','Receiver']], on = 'Receiver', how = 'left')
info2r_ow_inbound = info2r_ow_inbound.drop_duplicates('Sender')
info2r_ow_inbound = info2r_ow_inbound.set_index('Sender')


for title in ls_customer_ow:    
    data = inbound_report_tr_ow[inbound_report_tr_ow['Sender'] == title]
    table = [["סהכ",data["nis sum"].sum().round(),data['Remittance amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Receiver", "nis sum",'Remittance amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(" ")
    orderd = orderd.rename(columns = {'Sender':'Receiver1','Receiver':'Sender'})
    orderd = orderd.rename(columns = {'Receiver1':'Receiver','Source country':'Destination country'})
    orderd=change_names_and_order(location+r"/change_names.xlsx",orderd)
    
    doc = docx.Document(str(info2r_ow_inbound['report_name'].loc[title]) + "-" + str(info2r_ow_inbound['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow_inbound['report_name'].loc[title]) + "-" + str(info2r_ow_inbound['report_id'].loc[title]) + ".docx")
    